# Hyperparameter Tuning using HyperDrive

Import all the dependencies.

In [1]:
from azureml.core import Workspace, Experiment, Environment, ScriptRunConfig, Webservice, Dataset
from azureml.core.compute import ComputeTarget
from azureml.train.hyperdrive.sampling import BayesianParameterSampling
from azureml.train.hyperdrive.parameter_expressions import choice
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.widgets import RunDetails
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig, Model
from azureml.core.conda_dependencies import CondaDependencies
import joblib, requests, json, sklearn
import urllib.request

## Dataset

Access the data.

In [2]:
ws = Workspace.from_config()
experiment_name = 'Final_Project_HyperDrive'

experiment=Experiment(workspace=ws, name=experiment_name)

# log data
run = experiment.start_logging()

# Use existing compute target.
cluster_name = "Final-Project-Compute"
compute_cluster = ComputeTarget(workspace=ws, name=cluster_name)
compute_cluster.wait_for_completion(show_output=True)

# Load the dataset from the Workspace.
dataset_name = "heart_failure_clinical_records"
dataSet = Dataset.get_by_name(ws, dataset_name)
dataSet.take(5).to_pandas_dataframe()

Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75,0,582,0,20,1,265000,1.9,130,1,0,4,1
1,55,0,7861,0,38,0,263358,1.1,136,1,0,6,1
2,65,0,146,0,20,0,162000,1.3,129,1,1,7,1
3,50,1,111,0,20,0,210000,1.9,137,1,0,7,1
4,65,1,160,1,20,0,327000,2.7,116,0,0,8,1


## Hyperdrive Configuration

Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [3]:
# This is not required as we are using Bayesian sampling.
early_termination_policy = None

# different params that we will be using during training.
param_sampling = BayesianParameterSampling( {
        '--n_estimators': choice(range(10, 500)),
        '--min_samples_leaf': choice(range(10, 500)),
        '--max_features': choice('sqrt', 'log2'),
        '--oob_score': choice('True', 'False')
    })

# Setup environment for your training run
env = Environment.from_conda_specification(name='project_environment', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of training job
src = ScriptRunConfig(source_directory='.',
                      script='train.py',
                      # pass dataset as an input with name 'dataset'
                      arguments=['--input-data', dataSet.as_named_input('dataset')],
                      compute_target=compute_cluster,
                      environment=env)

# Create a HyperDriveConfig.
hyperdrive_run_config = HyperDriveConfig(run_config=src,
                             			 hyperparameter_sampling=param_sampling,
                                         policy=early_termination_policy,
                                         primary_metric_name="accuracy",
                                         primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                         max_total_runs=100,
                                         max_concurrent_runs=4)

In [4]:
# Submit the experiment.
hyperdrive_run = experiment.submit(hyperdrive_run_config, show_output=True)

## Run Details

Use the `RunDetails` widget to show the different experiments.

In [5]:
# Visualize all hyperparameter tuning runs.
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_f520b615-ab28-4ee5-a9b7-c201e50961dd
Web View: https://ml.azure.com/runs/HD_f520b615-ab28-4ee5-a9b7-c201e50961dd?wsid=/subscriptions/a0a76bad-11a1-4a2d-9887-97a29122c8ed/resourcegroups/aml-quickstarts-191366/workspaces/quick-starts-ws-191366&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

"<START>[2022-04-08T12:55:40.443094][API][INFO]Experiment created<END>\n""<START>[2022-04-08T12:55:41.192057][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n"<START>[2022-04-08T12:55:42.2587527Z][SCHEDULER][INFO]Scheduling job, id='HD_f520b615-ab28-4ee5-a9b7-c201e50961dd_0'<END><START>[2022-04-08T12:55:42.3122496Z][SCHEDULER][INFO]Scheduling job, id='HD_f520b615-ab28-4ee5-a9b7-c201e50961dd_1'<END><START>[2022-04-08T12:55:42.4250998Z][SCHEDULER][INFO]Scheduling job, id='HD_f520b615-ab28-4ee5-a9b7-c201e50961dd_2'<END><START>[2022-04-08T12:55:42.5564237Z][SCHEDULER][INFO]Scheduling job, id='HD_f520b615-ab28-4ee5-a9b7-c201e50961dd_3

{'runId': 'HD_f520b615-ab28-4ee5-a9b7-c201e50961dd',
 'target': 'Final-Project-Compute',
 'status': 'Completed',
 'startTimeUtc': '2022-04-08T12:55:40.20524Z',
 'endTimeUtc': '2022-04-08T13:46:12.774538Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name": "accuracy", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '2c88d8fe-6162-4c24-9684-911e2a7cb8dd',
  'user_agent': 'python/3.8.5 (Linux-5.4.0-1073-azure-x86_64-with-glibc2.10) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.39.0',
  'space_size': '960400',
  'score': '0.7666666666666667',
  'best_child_run_id': 'HD_f520b615-ab28-4ee5-a9b7-c201e50961dd_15',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://mlstrg191366.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_f520b615-ab28-4ee5-a9b7-c201e50961dd/azu

KeyError: 'log_files'

## Best Model

Get the best model from the hyperdrive experiments and display all the properties of the model.

In [6]:
# Best performing configuration and hyperparameter values.
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['accuracy'])
print('\n learning rate:',parameter_values[3])

Best Run Id:  HD_f520b615-ab28-4ee5-a9b7-c201e50961dd_15

 Accuracy: 0.7666666666666667

 learning rate: 295


In [7]:
# Save the best model.
# Register the model to deploy
model_hyperdrive = best_run.register_model(model_name='HyperDriveModel', model_path='outputs/HyperDriveModel.pkl')

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

Deploy the model as a Web Service on Azure Container Instance

In [8]:
#env = Environment(name="project_environment")
#env.python.conda_dependencies = CondaDependencies.create(pip_packages=[
    #'azureml-defaults',
    #'inference-schema[numpy-support]',
    #'joblib',
    #'numpy',
    #'scikit-learn=={}'.format(sklearn.__version__),
    #'scipy'
#])

# create an inference config.
#inference_config = InferenceConfig(entry_script='score.py', environment=env)

In [9]:
# create a deployment configuration (Azure Container Instances).
deployment_config = AciWebservice.deploy_configuration(
    cpu_cores=3, memory_gb=10, auth_enabled=True, enable_app_insights=True)

# deploy the model as a web service.
#service = Model.deploy(
    #workspace = ws,
    #name = "my-web-service",
    #models = [model_hyperdrive],
    #inference_config = inference_config,
    #deployment_config = deployment_config,
    #overwrite=True)

#service.wait_for_deployment(show_output=True)
#print(service.state)
#print(service.get_logs())

Send a request to the deployed web service to test it.

Now that the model is trained, run the test data through the trained model to get the predicted values. This calls the ACI web service to do the prediction.

Note that the JSON passed to the ACI web service is an array of rows of data. Each row should either be an array of values in the same order that was used for training or a dictionary where the keys are the same as the column names used for training.

In [10]:
#service = Webservice(workspace=ws, name="my-web-service")
#endpoint_url = service.scoring_uri

# If the service is authenticated, set the key or token
#key, _ = service.get_keys()

# Set the appropriate headers
#headers = {"Content-Type": "application/json"}
#headers["Authorization"] = f"Bearer {key}"

# Make the request and display the response and logs
# Correct format for input data : ["age", "anaemia", "creatinine_phosphokinase", "diabetes", "ejection_fraction", "high_blood_pressure", "platelets", "serum_creatinine", "serum_sodium", "sex", "smoking", "time"]
# Input : [49, 1, 80, 0, 30, 1, 427000, 1, 138, 0,	0, 12]. Output : 0
# Input : [82, 1, 379, 0, 50, 0, 47000, 1.3, 136, 1, 0, 13]. Output : 1

#test_data = [[49,1,80,0,30,1,427000,1,138,0,0,12],[82,1,379,0,50,0,47000,1.3,136,1,0,13]]
#data = json.dumps({"data": test_data})

#prediction = requests.post(endpoint_url, data, headers)
#prediction = service.run(data)

#print(prediction)
#prediction = json.loads(prediction)

#predict_result = json.loads(prediction.json())
#print(predict_result)

#for i in range(len(test_data)):
    #print ("\n If Input data is in form : age, anaemia, creatinine_phosphokinase, diabetes, ejection_fraction, high_blood_pressure, platelets, serum_creatinine, serum_sodium, sex, smoking, time : {}".format(test_data[i]))
    #print ("\n Then Chance of Death Event is : {}".format(prediction[i]))

Test web service

In [11]:
#if service.state == "Healthy":        
    #prediction = service.run(data)
    #print(prediction)
#else:
    #raise ValueError("Service deployment isn't healthy, can't call the service. Error: ", service.error)

Print the logs of the web service

In [12]:
#print(service.get_logs())

In [13]:
# delete the service, model, cluster.
#service.delete()
#model_hyperdrive.delete()
#compute_cluster.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

